# Group-By & RFM Analysis

- Recency: Did they make a purchase recently?
- Frequency: How often do they make a purchase?
- Monetary: How much do they spend?

In [1]:
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns


# Download Dataset or run in a kaggle notebook
# https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
df = pl.read_csv("2019-Nov.csv")

In [2]:
df.shape

(67501979, 9)

In [4]:
df.head().to_pandas()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,None,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [5]:
# working with dates docs: 
# https://pola-rs.github.io/polars-book/user-guide/howcani/timeseries/parsing_dates_times.html

df = df.with_column(
    pl.col("event_time").str.strptime(pl.Datetime, fmt="%Y-%m-%d %H:%M:%S %Z")
)

In [6]:
# Useful step to understand the data. Look at one user-id
df.filter(pl.col('user_id') == 520088904).to_pandas().sample(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
15,2019-11-01 00:43:52,view,1701552,2053013553031414015,computers.peripherals.monitor,acer,217.51,520088904,bbd7ce29-5406-4c52-b239-f0dd3b12427e
9,2019-11-01 00:38:37,view,1701552,2053013553031414015,computers.peripherals.monitor,acer,217.51,520088904,bbd7ce29-5406-4c52-b239-f0dd3b12427e
30,2019-11-15 19:12:07,view,4802960,2053013554658804075,electronics.audio.headphone,harper,20.28,520088904,bb7e87b8-0aa6-48b1-97a6-3178a2166532
24,2019-11-08 12:29:48,view,4800970,2053013554658804075,electronics.audio.headphone,sony,13.02,520088904,277ff36f-3e74-44b4-b573-03cf7ca4aa42
19,2019-11-03 07:55:11,view,1480706,2053013561092866779,computers.desktop,pulser,530.23,520088904,4a2763a0-a7ec-40e8-89f8-317b84f6658f


## 1. Keep rows with purchases only

In [7]:
df_purchases = df.filter(pl.col("event_type") == 'purchase')

In [8]:
df_purchases.shape

(916939, 9)

## 2. Let's compute RFM

In [9]:
# select unique values
df_purchases = df_purchases.select(['event_time', 'user_id', 'price']).unique()

In [10]:
df_purchases.shape

(916930, 3)

In [11]:
df_purchases.head()

event_time,user_id,price
datetime[μs],i64,f64
2019-11-01 00:00:41,559368633,566.3
2019-11-01 00:01:04,513351129,211.92
2019-11-01 00:04:51,562958505,128.42
2019-11-01 00:05:34,541854711,109.66
2019-11-01 00:06:33,557746614,488.8


### Compute time difference

In [12]:
from datetime import datetime

anchor_date = datetime(2019, 11, 30)

In [13]:
df_purchases = df_purchases.with_columns(
    (anchor_date - pl.col("event_time")).alias("date_diff") / (1e6 * 3600 * 24)
)

In [14]:
df_purchases.head()

event_time,user_id,price,date_diff
datetime[μs],i64,f64,f64
2019-11-01 00:00:41,559368633,566.3,28.999525
2019-11-01 00:01:04,513351129,211.92,28.999259
2019-11-01 00:04:51,562958505,128.42,28.996632
2019-11-01 00:05:34,541854711,109.66,28.996134
2019-11-01 00:06:33,557746614,488.8,28.995451


In [17]:
%%timeit
df_agg = df_purchases.groupby('user_id').agg([
    pl.col("date_diff").min().alias("recency"),
    pl.count().alias("frequency"),
    pl.col("price").sum().alias("monetary")
])

338 ms ± 32.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
df_agg.shape

(441638, 4)

In [19]:
df_agg.head()

user_id,recency,frequency,monetary
i64,f64,u32,f64
539459584,25.828484,1,134.62
513343488,24.38669,1,1422.31
526155776,21.678495,1,1029.37
568983552,-0.142176,2,602.5
545390592,17.638715,1,771.94


## 3. Convert to pandas for analysis and plotting

In [20]:
df_agg = df_agg.to_pandas()

In [21]:
breaks = np.arange(0, 1.1, 0.1)
num_vars = ['recency', 'frequency', 'monetary']
df_agg[num_vars].quantile(breaks)

,recency,frequency,monetary
0.0,-0.999815,1.0,0.770
0.1,1.381796,1.0,43.240
0.2,4.618451,1.0,84.660
0.3,8.523638,1.0,128.420
0.4,12.251690,1.0,176.850
0.5,12.492106,1.0,246.040
0.6,12.786208,1.0,339.720
0.7,15.641579,2.0,494.170
0.8,19.927676,2.0,795.474
0.9,24.442001,4.0,1364.000


In [22]:
num_vars = ['recency', 'frequency', 'monetary']
df_agg[num_vars].quantile([0, 0.25, 0.5, 0.75, 1])

,recency,frequency,monetary
0.00,-0.999815,1.0,0.7700
0.25,6.570278,1.0,108.6000
0.50,12.492106,1.0,246.0400
0.75,17.780564,2.0,616.9175
1.00,28.999525,519.0,203986.0700
